In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# GPU
device = torch.device("cuda:0")

# 训练集
trainset = torchvision.datasets.CIFAR10(root='./data_CIFAR10', train=True,
                                        download=False, transform=transform)
trainloader = DataLoader(trainset, batch_size=36)

# 测试集
testset = torchvision.datasets.CIFAR10(root='./data_CIFAR10', train=False,
                                       download=False, transform=transform)
testloader = DataLoader(trainset, batch_size=10000)

############################### 设置迭代器使得
test_data_iter = iter(testloader)
test_image, test_label = test_data_iter.next()
test_image = test_image.to(device)
test_label = test_label.to(device)

# 标签
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


# 创建网络
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 利用卷积层和最大池化来处理
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = LeNet()
net = net.to(device)

# 损失函数、
loss_function = nn.CrossEntropyLoss()
loss_function = loss_function.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 开始训练
for epoch in range(20):
    print("第{}轮训练开始".format(epoch + 1))
    # 计算损失
    running_loss = 0.0
    for step, data in enumerate(trainloader, start=0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 梯度清零
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        torch.save(net.state_dict(), "predict_{}".format(epoch))
        running_loss += loss.item()
        if step % 500 == 499:
            with torch.no_grad():
                outputs = net(test_image)
                predict_y = torch.max(outputs, dim=1)[1]
                accuracy = (predict_y == test_label).sum().item() / test_label.size(0)
                print("第{}轮训练 第{}次训练的损失值为：{}，准确率为：{}".format(epoch + 1, step + 1, running_loss,
                                                                              accuracy))
                running_loss = 0.0      
print("finish")

第1轮训练开始
第1轮训练 第500次训练的损失值为：868.898784995079，准确率为：0.4816
第1轮训练 第1000次训练的损失值为：699.795659840107，准确率为：0.5248
第2轮训练开始
第2轮训练 第500次训练的损失值为：594.548936188221，准确率为：0.6154
第2轮训练 第1000次训练的损失值为：552.6333025693893，准确率为：0.6184
第3轮训练开始
第3轮训练 第500次训练的损失值为：503.1597937345505，准确率为：0.6735
第3轮训练 第1000次训练的损失值为：478.1283648312092，准确率为：0.6644
第4轮训练开始
第4轮训练 第500次训练的损失值为：449.8695732355118，准确率为：0.7108
第4轮训练 第1000次训练的损失值为：432.21887215971947，准确率为：0.6971
第5轮训练开始
第5轮训练 第500次训练的损失值为：409.17535853385925，准确率为：0.7259
第5轮训练 第1000次训练的损失值为：396.80805492401123，准确率为：0.7144
第6轮训练开始
第6轮训练 第500次训练的损失值为：378.40518367290497，准确率为：0.7411
第6轮训练 第1000次训练的损失值为：365.3074653148651，准确率为：0.7296
第7轮训练开始
第7轮训练 第500次训练的损失值为：352.73629531264305，准确率为：0.7608
第7轮训练 第1000次训练的损失值为：341.0641538500786，准确率为：0.7243
第8轮训练开始
第8轮训练 第500次训练的损失值为：329.25546258687973，准确率为：0.7658
第8轮训练 第1000次训练的损失值为：317.81398214399815，准确率为：0.7487
第9轮训练开始
第9轮训练 第500次训练的损失值为：306.8342633843422，准确率为：0.7667
第9轮训练 第1000次训练的损失值为：303.3025632351637，准确率为：0.7527
第10轮训练开始
第10轮训练 第500次训练的损失值为：284.